# Dynamic Routing

Bridgic supports dynamic routing to the corresponding worker based on runtime conditions. Now let's understand it with a sample example.

<br>
<div style="text-align: center;">
<img src="../../../imgs/dynamic_router.png" alt="Parameter Passing" width="400" height="250">
</div>
<br>

## Domain chatbot 

Dynamic routing is a common requirement in chatbots. For instance, it involves analyzing the input query to determine the main domain or field it pertains to, and then dynamically routing it to the corresponding processing logic.

The user inputs the original query, we identify whether it is a medical-related or legal-related issue, and then we answer it with specialized logic. There are three steps:

1. Receive user input
2. Domain recognition and dynamic routing to correct worker
3. Answer query

### 1. Initialize

Before we start, let's prepare the running environment.

In [ ]:
# Get the environment variables.
import os
from dotenv import load_dotenv
load_dotenv()

_api_base = os.environ.get("VLLM_SERVER_API_BASE")
_api_key = os.environ.get("VLLM_SERVER_API_KEY")
_model_name = os.environ.get("VLLM_SERVER_MODEL_NAME")

# Import the necessary packages.
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.protocols import Regex
from bridgic.core.model.types import Message, Role
from bridgic.llms.vllm.vllm_server_llm import VllmServerLlm

llm = VllmServerLlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    timeout=20,
)

### 2. Complete

We assume that the user query we receive is a string. Let's implement the domain chatbot to answer the user query.

In [4]:
class DomainChatbot(GraphAutoma):
    @worker(is_start=True)
    async def pre_query(self, query: str):  # receive user input and preprocess it
        return query

    @worker(dependencies=["pre_query"])
    async def domain_recognition(self, query: str):  # domain recognition and dynamic routing
        response: str = await llm.astructured_output(
            model=_model_name,
            constraint=Regex(pattern=r"^medical$|^legal$", description="model output should be only 'medical' or 'legal'"),
            messages=[
                Message.from_text(text="Please identify the domain of the following query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        print(f'The query domain is: {response}')  # print the response
        
        # dynamic routing
        if response == "medical":
            return self.ferry_to('answer_medical_query', query)
        elif response == "legal":
            return self.ferry_to('answer_legal_query', query)
    
    @worker(is_output=True)
    async def answer_medical_query(self, query: str):  # answer medical-related query
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a medical expert. Answer the following medical-related query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        return response.message.content

    @worker(is_output=True)
    async def answer_legal_query(self, query: str):  # answer legal-related query
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a legal expert. Answer the following legal-related query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        return response.message.content

Let's run it!

In [5]:
query = "What is the aspirin?"
domain_chatbot = DomainChatbot()
await domain_chatbot.arun(query)

The query domain is: medical


'Aspirin, also known as acetylsalicylic acid, is a widely used medication with several important medical properties. It is one of the oldest and most well-known over-the-counter (OTC) drugs, originally derived from willow bark.\n\n### Key Uses of Aspirin:\n\n1. **Pain Relief (Analgesic)**  \n   Aspirin helps relieve mild to moderate pain such as headaches, toothaches, muscle aches, and menstrual cramps.\n\n2. **Reducing Fever (Antipyretic)**  \n   It lowers body temperature in cases of fever, such as in infections.\n\n3. **Reducing Inflammation (Anti-inflammatory)**  \n   Aspirin helps reduce inflammation, which can be beneficial in conditions like rheumatoid arthritis or other inflammatory disorders.\n\n4. **Preventing Blood Clots (Antiplatelet Effect)**  \n   This is one of its most important therapeutic uses. Aspirin inhibits the formation of blood clots by blocking the action of a substance called cyclooxygenase (COX), which reduces the production of thromboxane—a molecule that pro

> If you encounter a Readtimeout error during the execution of the above process, it might be because the timeout period set during the initialization of the llm is too short, causing the model to time out before it finishes responding.

Great! We have successfully completed the domain chatbot.

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

## What have we done?

We have implemented the routing mechanism using Bridgic.

### Ferry To

In Bridgic, we use the `ferry_to` mechanism to achieve dynamic routing. Its first parameter is the `key` of the worker to which the route will be directed, and the second parameter is the input parameter passed to the target worker. 